In [1]:
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer, recall_score, f1_score, accuracy_score
import numpy as np
from imblearn.metrics import geometric_mean_score
import pandas as pd
from collections import Counter
from tqdm import tqdm

In [2]:
predictions = pd.read_csv('unlabeled_predictions_2.csv')
vt_labels = pd.read_csv("vt_labels.csv")
#labels_1 = pd.read_csv("vt_labels_1.csv").Labels.values.tolist()

In [3]:
def specificity(y_true, y_pred): 
    tn = confusion_matrix(y_true, y_pred)[0, 0]
    fp = confusion_matrix(y_true, y_pred)[0, 1]
    neg = tn + fp
    return tn/neg


In [6]:
len(vt_labels)

179117

In [22]:
predictions = pd.concat([vt_labels.Domains, predictions], axis=1)
predictions.to_csv('unlabeled_predictions_2_doms.csv', index=False, header=True)

In [23]:
predictions = predictions[:55055]

In [8]:
predictions.drop(vt_labels[vt_labels.Label == 'not_found'].index, inplace=True)
vt_labels.drop(vt_labels[vt_labels.Label == 'not_found'].index, inplace=True)

In [9]:
predictions.reset_index(drop=True, inplace=True)
vt_labels.reset_index(drop=True, inplace=True)

In [10]:
vt_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88679 entries, 0 to 88678
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Domain  88679 non-null  object
 1   Label   88679 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [12]:
labels = vt_labels.Label.values.tolist()
labels = [int(i) for i in labels]

In [13]:
predictions.head()

,Labeled dtc,Labeled rfc,Labeled xgb,Labeled gbc,Labeled mlp,Labeled svc,Labeled dtc_sd,Labeled rfc_sd,Labeled xgb_sd,Labeled gbc_sd,Labeled mlp_sd,Labeled svc_sd
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,1,1,0,1,0,1
3,0,0,1,0,1,0,1,1,1,1,0,1
4,1,1,1,1,1,1,1,1,1,1,1,1


In [28]:
#ht_sd = predictions['Labeled with subdomains'].values.tolist()
#ht_nosd = predictions['Labeled w/o subdomains'].values.tolist()
dtc = predictions['Labeled dtc'].values.tolist()
rfc = predictions['Labeled rfc'].values.tolist()
xgb = predictions['Labeled xgb'].values.tolist()
gbc = predictions['Labeled gbc'].values.tolist()
mlp = predictions['Labeled mlp'].values.tolist()
svc = predictions['Labeled svc'].values.tolist()

dtc_sd = predictions['Labeled dtc_sd'].values.tolist()
rfc_sd = predictions['Labeled rfc_sd'].values.tolist()
xgb_sd = predictions['Labeled xgb_sd'].values.tolist()
gbc_sd = predictions['Labeled gbc_sd'].values.tolist()
mlp_sd = predictions['Labeled mlp_sd'].values.tolist()
svc_sd = predictions['Labeled svc_sd'].values.tolist()

In [29]:
predictions.drop('Domains', inplace=True, axis=1)

In [37]:
print(Counter(labels))

Counter({1: 5719, 0: 5563})


In [33]:
len(labels)

11282

In [ ]:
print(Counter(labels))
print(Counter(ht_nosd[:74553]))

In [ ]:
print(Counter(ht_sd))

In [ ]:
print(specificity(labels, ht_sd[:74553]))
print(recall_score(labels, ht_sd[:74553]))

In [ ]:
print(specificity(labels, ht_nosd[:74553]))
print(recall_score(labels, ht_nosd[:74553]))

In [38]:
print(specificity(labels, xgb[:11282]))
print(recall_score(labels, xgb[:11282]))

0.6349092216429983
0.8102815177478581


In [39]:
print(specificity(labels, xgb_sd[:11282]))
print(recall_score(labels, xgb_sd[:11282]))

0.6232248786625921
0.8300402168211226


In [ ]:
confusion_matrix(labels, ht_sd[:74553])

In [ ]:
confusion_matrix(labels, ht_nosd[:74553])

In [40]:
confusion_matrix(labels, dtc[:11282])

array([[3508, 2055],
       [1681, 4038]], dtype=int64)

In [41]:
confusion_matrix(labels, rfc[:11282])

array([[3603, 1960],
       [1227, 4492]], dtype=int64)

In [42]:
confusion_matrix(labels, xgb[:11282])

array([[3532, 2031],
       [1085, 4634]], dtype=int64)

In [43]:
confusion_matrix(labels,gbc[:11282])

array([[3450, 2113],
       [1034, 4685]], dtype=int64)

In [44]:
confusion_matrix(labels, mlp[:11282])

array([[3689, 1874],
       [1732, 3987]], dtype=int64)

In [45]:
confusion_matrix(labels, svc[:11282])

array([[3672, 1891],
       [1809, 3910]], dtype=int64)

In [46]:
confusion_matrix(labels, dtc_sd[:11282])

array([[3591, 1972],
       [ 925, 4794]], dtype=int64)

In [47]:
cohen_kappa_score(labels, dtc_sd[:11282])

0.4850184496449468

In [48]:
confusion_matrix(labels, rfc_sd[:11282])

array([[3611, 1952],
       [1046, 4673]], dtype=int64)

In [49]:
confusion_matrix(labels, xgb_sd[:11282])

array([[3467, 2096],
       [ 972, 4747]], dtype=int64)

In [50]:
confusion_matrix(labels, gbc_sd[:11282])

array([[3661, 1902],
       [1052, 4667]], dtype=int64)

In [51]:
confusion_matrix(labels, mlp_sd[:11282])

array([[3649, 1914],
       [1299, 4420]], dtype=int64)

In [52]:
confusion_matrix( labels, svc_sd[:11282])

array([[3437, 2126],
       [ 933, 4786]], dtype=int64)

In [53]:
def calc_kappa(y_1, df):
    k_s = []
    for i in df.columns:
        k_s.append(cohen_kappa_score(y_1, df))
    return k_s

In [54]:
Counter(labels)

Counter({1: 5719, 0: 5563})

In [61]:
kk_0 = []
for i in predictions.columns:
    kk_0.append(geometric_mean_score(labels, predictions[i].values.tolist()[:11282]))
    

kk_0 = pd.Series(kk_0, name='Recall score against VT labels')

i = 0
for j in predictions.columns:
    kk_0.rename(index={i: j}, inplace=True)
    i += 1

In [62]:
pd.concat([kk_0], axis=1)

,Recall score against VT labels
Labeled dtc,0.667265
Labeled rfc,0.713243
Labeled xgb,0.717255
Labeled gbc,0.712771
Labeled mlp,0.679928
Labeled svc,0.671777
Labeled dtc_sd,0.735601
Labeled rfc_sd,0.728278
Labeled xgb_sd,0.719237
Labeled gbc_sd,0.732832


In [ ]:
kk_1 = []
for i in predictions.columns:
    kk_1.append(cohen_kappa_score(ht_sd, predictions[i].values.tolist()[:30553]))

kk_1 = pd.Series(kk_1, name='HT_sd')

In [18]:
kk_2 = []
for i in predictions.columns:
    kk_2.append(cohen_kappa_score(ht_nosd, predictions[i].values.tolist()[:30553]))

kk_2 = pd.Series(kk_2, name='HT_nosd')

In [19]:
kk_3 = []
for i in predictions.columns:
    kk_3.append(cohen_kappa_score(dtc, predictions[i].values.tolist()[:30553]))
    
kk_3 = pd.Series(kk_3, name='DTC_nosd')

In [20]:
kk_4 = []
for i in predictions.columns:
    kk_4.append(cohen_kappa_score(rfc, predictions[i].values.tolist()[:30553]))
    
kk_4 = pd.Series(kk_4, name='RFC_nosd')

In [21]:
kk_5 = []
for i in predictions.columns:
    kk_5.append(cohen_kappa_score(xgb, predictions[i].values.tolist()[:30553]))
kk_5 = pd.Series(kk_5, name='XGB_nosd')

In [22]:
kk_6 = []
for i in predictions.columns:
    kk_6.append(cohen_kappa_score(gbc, predictions[i].values.tolist()[:30553]))
kk_6 = pd.Series(kk_6, name='GBC_nosd')

In [23]:
kk_7 = []
for i in predictions.columns:
    kk_7.append(cohen_kappa_score(mlp, predictions[i].values.tolist()[:30553]))
kk_7 = pd.Series(kk_7, name='MLP_nosd')

In [24]:
kk_8 = []
for i in predictions.columns:
    kk_8.append(cohen_kappa_score(svc, predictions[i].values.tolist()[:30553]))
kk_8 = pd.Series(kk_8, name='SVC_nosd')

In [25]:
kk_9 = []
for i in predictions.columns:
    kk_9.append(cohen_kappa_score(dtc_sd, predictions[i].values.tolist()[:30553]))
kk_9 = pd.Series(kk_9, name='DTC_sd')

In [26]:
kk_10 = []
for i in predictions.columns:
    kk_10.append(cohen_kappa_score(rfc_sd, predictions[i].values.tolist()[:30553]))
kk_10 = pd.Series(kk_10, name='RFC_sd')

In [27]:
kk_11 = []
for i in predictions.columns:
    kk_11.append(cohen_kappa_score(xgb_sd, predictions[i].values.tolist()[:30553]))
kk_11 = pd.Series(kk_11, name='XGB_sd')

In [28]:
kk_12 = []
for i in predictions.columns:
    kk_12.append(cohen_kappa_score(gbc_sd, predictions[i].values.tolist()[:30553]))
kk_12 = pd.Series(kk_12, name='GBC_sd')

In [29]:
kk_13 = []
for i in predictions.columns:
    kk_13.append(cohen_kappa_score(mlp_sd, predictions[i].values.tolist()[:30553]))
kk_13 = pd.Series(kk_13, name='MLP_sd')

In [30]:
kk_14 = []
for i in predictions.columns:
    kk_14.append(cohen_kappa_score(svc_sd, predictions[i].values.tolist()[:30553]))
kk_14 = pd.Series(kk_14, name='SVC_sd')

In [31]:
scores = pd.concat([kk_1, kk_2, kk_3, kk_4, kk_5, kk_6, kk_7, kk_8, kk_9, kk_10, kk_11, kk_12, kk_13, kk_14], axis=1)

In [32]:
i = 0
for j in scores.columns:
    scores.rename(index={i: j}, inplace=True)
    i += 1

In [33]:
# at 30553
scores

,HT_sd,HT_nosd,DTC_nosd,RFC_nosd,XGB_nosd,GBC_nosd,MLP_nosd,SVC_nosd,DTC_sd,RFC_sd,XGB_sd,GBC_sd,MLP_sd,SVC_sd
HT_sd,1.000000,0.733355,0.777624,0.845932,0.851883,0.836810,0.906817,0.859882,0.951362,0.897768,0.891480,0.909447,0.912916,0.953949
HT_nosd,0.733355,1.000000,0.695464,0.761841,0.753023,0.804358,0.774141,0.784409,0.770997,0.738959,0.742047,0.744479,0.759810,0.773056
DTC_nosd,0.777624,0.695464,1.000000,0.849142,0.848101,0.807258,0.820421,0.787973,0.819595,0.845113,0.833160,0.828802,0.810710,0.819461
RFC_nosd,0.845932,0.761841,0.849142,1.000000,0.923542,0.891981,0.900799,0.880820,0.891123,0.928202,0.911579,0.894432,0.889071,0.890466
XGB_nosd,0.851883,0.753023,0.848101,0.923542,1.000000,0.889695,0.903178,0.862273,0.897205,0.919874,0.934669,0.903853,0.895275,0.898013
GBC_nosd,0.836810,0.804358,0.807258,0.891981,0.889695,1.000000,0.891212,0.869533,0.880197,0.870561,0.879799,0.886916,0.877941,0.880965
MLP_nosd,0.906817,0.774141,0.820421,0.900799,0.903178,0.891212,1.000000,0.912124,0.951643,0.924997,0.924674,0.935677,0.969362,0.952391
SVC_nosd,0.859882,0.784409,0.787973,0.880820,0.862273,0.869533,0.912124,1.000000,0.901736,0.873408,0.875313,0.887197,0.906824,0.904329
DTC_sd,0.951362,0.770997,0.819595,0.891123,0.897205,0.880197,0.951643,0.901736,1.000000,0.944686,0.937542,0.955857,0.959787,0.996948
RFC_sd,0.897768,0.738959,0.845113,0.928202,0.919874,0.870561,0.924997,0.873408,0.944686,1.000000,0.954896,0.944292,0.935728,0.944476


In [23]:
# at 10000
scores

,HT_sd,HT_nosd,DTC_nosd,RFC_nosd,XGB_nosd,GBC_nosd,MLP_nosd,SVC_nosd,DTC_sd,RFC_sd,XGB_sd,GBC_sd,MLP_sd,SVC_sd
HT_sd,1.000000,0.472702,0.536007,0.635022,0.641643,0.622850,0.759843,0.602941,0.833846,0.720133,0.717321,0.742215,0.777479,0.833846
HT_nosd,0.472702,1.000000,0.488778,0.538546,0.489606,0.579053,0.529465,0.631339,0.543579,0.457043,0.468487,0.474360,0.515412,0.543579
DTC_nosd,0.536007,0.488778,1.000000,0.676990,0.694215,0.661483,0.612966,0.547179,0.646421,0.692239,0.694215,0.676805,0.607393,0.646421
RFC_nosd,0.635022,0.538546,0.676990,1.000000,0.841810,0.874899,0.761807,0.725382,0.763219,0.828391,0.808507,0.783177,0.752192,0.763219
XGB_nosd,0.641643,0.489606,0.694215,0.841810,1.000000,0.817004,0.787290,0.647358,0.782216,0.838146,0.859107,0.800113,0.771227,0.782216
GBC_nosd,0.622850,0.579053,0.661483,0.874899,0.817004,1.000000,0.744904,0.728654,0.747071,0.749378,0.756915,0.754506,0.730170,0.747071
MLP_nosd,0.759843,0.529465,0.612966,0.761807,0.787290,0.744904,1.000000,0.738204,0.919553,0.841608,0.838340,0.883735,0.960071,0.919553
SVC_nosd,0.602941,0.631339,0.547179,0.725382,0.647358,0.728654,0.738204,1.000000,0.711062,0.636694,0.633619,0.660827,0.725705,0.711062
DTC_sd,0.833846,0.543579,0.646421,0.763219,0.782216,0.747071,0.919553,0.711062,1.000000,0.876638,0.873632,0.900143,0.941758,1.000000
RFC_sd,0.720133,0.457043,0.692239,0.828391,0.838146,0.749378,0.841608,0.636694,0.876638,1.000000,0.929761,0.910755,0.860261,0.876638


In [70]:
# at 350000 (original)
scores

,HT_sd,HT_nosd,DTC_nosd,RFC_nosd,XGB_nosd,GBC_nosd,MLP_nosd,SVC_nosd,DTC_sd,RFC_sd,XGB_sd,GBC_sd,MLP_sd,SVC_sd
HT_sd,1.000000,0.380922,0.223846,0.219961,0.230166,0.205385,0.251964,0.234591,0.282199,0.249172,0.247880,0.247925,0.250202,0.283565
HT_nosd,0.380922,1.000000,0.168401,0.182236,0.189823,0.174776,0.196362,0.194510,0.206367,0.196916,0.197348,0.199779,0.202501,0.206662
DTC_nosd,0.223846,0.168401,1.000000,0.840180,0.833158,0.772407,0.801836,0.786061,0.796798,0.817793,0.808683,0.803868,0.786245,0.796271
RFC_nosd,0.219961,0.182236,0.840180,1.000000,0.925738,0.858297,0.897131,0.890813,0.879436,0.912516,0.898605,0.878642,0.875025,0.879449
XGB_nosd,0.230166,0.189823,0.833158,0.925738,1.000000,0.843979,0.915568,0.892551,0.899669,0.916257,0.930267,0.896692,0.892036,0.899631
GBC_nosd,0.205385,0.174776,0.772407,0.858297,0.843979,1.000000,0.848544,0.852477,0.825274,0.817670,0.821906,0.826516,0.823985,0.825561
MLP_nosd,0.251964,0.196362,0.801836,0.897131,0.915568,0.848544,1.000000,0.945947,0.957674,0.925052,0.928809,0.931833,0.956917,0.958294
SVC_nosd,0.234591,0.194510,0.786061,0.890813,0.892551,0.852477,0.945947,1.000000,0.934562,0.907943,0.906774,0.917559,0.937460,0.935943
DTC_sd,0.282199,0.206367,0.796798,0.879436,0.899669,0.825274,0.957674,0.934562,1.000000,0.952813,0.950820,0.956459,0.967228,0.998285
RFC_sd,0.249172,0.196916,0.817793,0.912516,0.916257,0.817670,0.925052,0.907943,0.952813,1.000000,0.962518,0.948853,0.942135,0.952739
